## **2A.** When is the best time of day, day of the week, and time of year to fly to minimise delays?

## To access the project files to run this notebook

Check github (link: https://github.com/inc0gn1k0/submission_final_st2195.git) to access the entire project folder.

## **2A** When is the best time of day, day of the week, and time of year to fly to minimise delays?

```
Analysis Recipe:
1. For Loop
2. lists
3 3 SQLite queries picking out data for when Delays are zero.
4. head() function
5. concatenation step

```

In [1]:
import sqlite3 #ships with python
import pandas as pd #should be already installed in your env
import dataframe_image as dfi #should be already installed in your env
import os #ships with python


current_dir = os.getcwd()
pathname = os.path.join(current_dir, '..', 'raw_data', 'comp97to07.db')
conn = sqlite3.connect(pathname)

'''
(Update 26/Jan/2024: we still need to print each table and highlight the modal rows in each, so that the final report shows our answer clearly)

Write a query that checks each table in the database comp97to07 for the best time of day to minimise delays. 
Repeat this query for each table (each year and go with the lowest ArrDelay and DepDelay)

For each table what is the CRSDepTime with the most frequent incidence of ArrDelay and DepDelay of 0, what is the day of the week with the most frequent incidence of ArrDelay and DepDelay of 0, what is the month with the most freq incidence of ArrDelay and DepDelay

We check the variable_description.csv to understand the naming system of the columns
'''

# Mapping dictionaries :)
strings_of_the_week = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
months_of_the_year = {'January': 1, 'February':2, 'March':3,'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

#Best Time of Day

subset_of_years = [1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]
top_rows_time = []

for i, years in enumerate(subset_of_years, start=1998):

    time_of_day = f'''
    SELECT Year, CRSDepTime, COUNT(CRSDepTime) AS Frequency, ArrDelay, DepDelay
    FROM Y{i}
    WHERE DepDelay = 0 AND ArrDelay = 0
    GROUP BY CRSDepTime
    ORDER BY Frequency DESC
    '''

    result = conn.execute(time_of_day)
    time_of_day_df = pd.DataFrame(result.fetchall(), columns=[desc[0] for desc in result.description])

    top_rows_time.append(time_of_day_df.head(1))


final_df_time = pd.concat(top_rows_time, ignore_index=True)
final_df_time['CRSDepTime'] = final_df_time['CRSDepTime'].astype(int)

#print(time_of_day_df.head(5))

print(final_df_time) 

'''
Hence the best time of day to minimise arrival delays and departure delays in all 4 years is to fly at 7:00AM
'''

#Best day of week

top_rows_week = []

for i, years in enumerate(subset_of_years, start=1998):

    day_of_week = f'''
    SELECT Year, DayOfWeek, COUNT(DayOfWeek) AS Frequency, ArrDelay, DepDelay
    FROM Y{i}
    WHERE DepDelay = 0 AND ArrDelay = 0
    GROUP BY DayOfWeek
    ORDER BY Frequency DESC
    '''

    result = conn.execute(day_of_week)
    day_of_week_df = pd.DataFrame(result.fetchall(), columns=[desc[0] for desc in result.description])

    top_rows_week.append(day_of_week_df.head(1))


final_df_week = pd.concat(top_rows_week, ignore_index=True)
final_df_week['DayOfWeek'] = final_df_week['DayOfWeek'].map({v: k for k, v in strings_of_the_week.items()})

#print(day_of_week_df.head(5))

print(final_df_week) 
'''
Hence the best day of the week to minimise arrival delays and departure delays 
in all 4 years is to fly on a Tuesday, the 2nd day of the week...
'''

#Best time of year (which month and hence which season)

top_rows_month = []

for i, years in enumerate(subset_of_years, start=1998):

    time_of_year = f'''
    SELECT Year, Month, COUNT(Month) AS Frequency, ArrDelay, DepDelay
    FROM Y{i}
    WHERE DepDelay = 0 AND ArrDelay = 0
    GROUP BY Month
    ORDER BY Frequency DESC
    '''

    result = conn.execute(time_of_year)
    time_of_year_df = pd.DataFrame(result.fetchall(), columns=[desc[0] for desc in result.description])

    top_rows_month.append(time_of_year_df.head(1))

final_df_month = pd.concat(top_rows_month, ignore_index=True)
final_df_month['Month'] = final_df_month['Month'].map({v: k for k, v in months_of_the_year.items()})

#print(day_of_week_df.head(5))

print(final_df_month) 

'''
Hence the best time of year to minimise arrival delays and departure delays 
in all 4 years is to fly in the fourth month in the year, April

This time of year is known as "Spring"...so travelers should fly in April, in the Spring.
'''

conn.close()

# Slightly complicated function for styling the modal cells, I decied to use a lambda function as it allows us to embed function logic within a function!
def highlight_mode(df):
    '''Highlight the mode(s) in each column yellow.'''
    attrs = []
    for column in df.columns:
        mode_vals = df[column].mode().tolist()  # Get the mode values as a list
        attrs.append(df[column].apply(lambda x: 'background-color: yellow' if x in mode_vals else ''))
    return pd.DataFrame(attrs).T  # Transpose since I iterated by columns and want the formatting to be applied on a row by row basis....

# Apply the highlighting function and save the styled DataFrame as an image
styled_final_df_time = final_df_time.style.apply(highlight_mode,subset=['CRSDepTime'],axis=None)
styled_final_df_week = final_df_week.style.apply(highlight_mode,subset=['DayOfWeek'],axis=None)
styled_final_df_month = final_df_month.style.apply(highlight_mode,subset=['Month'],axis=None)

if not os.path.exists('./py_images'):
    os.makedirs('./py_images')

# Save styled DataFrames as images
dfi.export(styled_final_df_time, './py_images/2A_final_df_time.png')
dfi.export(styled_final_df_week, './py_images/2A_final_df_week.png')
dfi.export(styled_final_df_month, './py_images/2A_final_df_month.png')


   Year  CRSDepTime  Frequency  ArrDelay  DepDelay
0  1998         700       2148       0.0       0.0
1  1999         700       1884       0.0       0.0
2  2000         700       2129       0.0       0.0
3  2001         700       2024       0.0       0.0
4  2002         700       1802       0.0       0.0
5  2003         630       2330       0.0       0.0
6  2004         700       2155       0.0       0.0
7  2005         700       1788       0.0       0.0
8  2006         700       1085       0.0       0.0
9  2007         600        670       0.0       0.0
   Year  DayOfWeek  Frequency  ArrDelay  DepDelay
0  1998    Tuesday      17308       0.0       0.0
1  1999    Tuesday      17169       0.0       0.0
2  2000    Tuesday      16009       0.0       0.0
3  2001     Monday      15693       0.0       0.0
4  2002    Tuesday      14174       0.0       0.0
5  2003     Monday      27519       0.0       0.0
6  2004  Wednesday      19791       0.0       0.0
7  2005    Tuesday      17899       0.0